In [1]:
import resource
rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (4096, rlimit[1]))

import sys
import torch
from sagemakercv.detection.detector import build_detection_model
from utils import (unarchive_data, 
                   get_training_world, 
                   is_sm, 
                   is_sm_dist, 
                   get_herring_world)
from sagemakercv.training import make_optimizer, make_lr_scheduler
from sagemakercv.data import make_data_loader, Prefetcher
from sagemakercv.utils.checkpoint import DetectronCheckpointer
from sagemakercv.utils.runner import build_hooks, Runner
from sagemakercv.utils.runner.hooks.checkpoint import DetectronCheckpointHook
from sagemakercv.training.trainers import train_step
import apex
from configs import cfg
from tqdm.auto import tqdm
from statistics import mean

if (torch._C, '_jit_set_profiling_executor') :
    torch._C._jit_set_profiling_executor(False)
if (torch._C, '_jit_set_profiling_mode') :
    torch._C._jit_set_profiling_mode(False)
    
torch.backends.cuda.matmul.allow_tf32 = True

In [2]:
cfg.merge_from_file('configs/ec2_mrcnn_bs8_O1.yaml')

In [3]:
train_coco_loader, num_iterations = make_data_loader(cfg, is_distributed=False)

NOTE! Installing ujson may make loading annotations faster.
loading annotations into memory...
Done (t=16.04s)
creating index...
index created!
shapes=None
passthrough=False


In [4]:
device = torch.device(cfg.MODEL.DEVICE)
train_iterator = Prefetcher(iter(train_coco_loader), device)

[2021-09-08 13:05:16.003 smcv-pt-1-9-ml-g4dn-12xlarge-df425b7e73d0c422dd55a6882a7b:1233 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-09-08 13:05:16.032 smcv-pt-1-9-ml-g4dn-12xlarge-df425b7e73d0c422dd55a6882a7b:1233 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [5]:
model = build_detection_model(cfg)
model.to(device)
optimizer = make_optimizer(cfg, model)
scheduler = make_lr_scheduler(cfg, optimizer)

In [6]:
hooks = build_hooks(cfg)

In [7]:
is_fp16 = (cfg.OPT_LEVEL == "O4")
if is_fp16:
    # convert model to FP16
    model.half()

In [8]:
runner = Runner(model, train_step, cfg, device, optimizer, scheduler)

In [9]:
for hook in hooks:
    runner.register_hook(hook, priority='HIGHEST' if isinstance(hook, DetectronCheckpointHook) else 'NORMAL')

In [ ]:
runner.run(train_iterator, num_iterations)

Start running, work_dir: /root/model_outputs


INFO:sagemakercv.utils.runner.runner:Start running, work_dir: /root/model_outputs


max: 2 epochs


INFO:sagemakercv.utils.runner.runner:max: 2 epochs
INFO:sagemakercv.utils.checkpoint:Loading checkpoint from /home/sagemaker-user/data/weights/R-50.pkl
INFO:sagemakercv.utils.c2_model_loading:Remapping C2 weights
INFO:sagemakercv.utils.c2_model_loading:C2 name: conv1_b              mapped name: conv1.bias
INFO:sagemakercv.utils.c2_model_loading:C2 name: conv1_w              mapped name: conv1.weight
INFO:sagemakercv.utils.c2_model_loading:C2 name: fc1000_b             mapped name: fc1000.bias
INFO:sagemakercv.utils.c2_model_loading:C2 name: fc1000_w             mapped name: fc1000.weight
INFO:sagemakercv.utils.c2_model_loading:C2 name: res2_0_branch1_b     mapped name: layer1.0.downsample.0.bias
INFO:sagemakercv.utils.c2_model_loading:C2 name: res2_0_branch1_bn_b  mapped name: layer1.0.downsample.1.bias
INFO:sagemakercv.utils.c2_model_loading:C2 name: res2_0_branch1_bn_s  mapped name: layer1.0.downsample.1.weight
INFO:sagemakercv.utils.c2_model_loading:C2 name: res2_0_branch1_w     map

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Start time: 2021-09-08 13:05:19.205408


...

INFO:sagemakercv.utils.runner.runner:Epoch [1][2450/14659]	lr: 0.01742, eta: 5:23:50, step time: 1.383, loss_classifier: 0.3464, loss_box_reg: 0.2222, loss_carl: 0.2136, loss_mask: 0.4040, loss_objectness: 0.2410, loss_rpn_box_reg: 0.0248, total_loss: 1.4519


Epoch [1][2500/14659]	lr: 0.01739, eta: 5:23:15, step time: 1.385, loss_classifier: 0.3736, loss_box_reg: 0.2397, loss_carl: 0.2302, loss_mask: 0.3944, loss_objectness: 0.2370, loss_rpn_box_reg: 0.0188, total_loss: 1.4938


INFO:sagemakercv.utils.runner.runner:Epoch [1][2500/14659]	lr: 0.01739, eta: 5:23:15, step time: 1.385, loss_classifier: 0.3736, loss_box_reg: 0.2397, loss_carl: 0.2302, loss_mask: 0.3944, loss_objectness: 0.2370, loss_rpn_box_reg: 0.0188, total_loss: 1.4938


Epoch [1][2550/14659]	lr: 0.01736, eta: 5:27:19, step time: 1.408, loss_classifier: 0.3721, loss_box_reg: 0.2331, loss_carl: 0.2229, loss_mask: 0.4048, loss_objectness: 0.2377, loss_rpn_box_reg: 0.0213, total_loss: 1.4919


INFO:sagemakercv.utils.runner.runner:Epoch [1][2550/14659]	lr: 0.01736, eta: 5:27:19, step time: 1.408, loss_classifier: 0.3721, loss_box_reg: 0.2331, loss_carl: 0.2229, loss_mask: 0.4048, loss_objectness: 0.2377, loss_rpn_box_reg: 0.0213, total_loss: 1.4919
